---

# Un arc-en-ciel de couleurs dans tes yeux!
### GoPiGo 101 - Série d'exercices 6
##### Manipulation des **yeux**.

Les yeux sont les deux diodes électroluminescentes (DEL ou _LED_ en anglais) de couleur configurable se trouvant au-dessus de la carte rouge `Dexter GoPiGo3`. Ces DEL représentent les yeux du robot `Dexter` imprimé en blanc au-dessus de la carte. Malheureusement, ils se trouvent partiellement obstrués par le câble de la caméra.

---

Les fonctions suivantes permettent la manipulation des yeux :
 - pour l'oeil gauche :
     - `EasyGoPiGo3.set_left_eye_color` : détermine la couleur de l'oeil gauche
     - `EasyGoPiGo3.close_left_eye` : éteint l'oeil gauche
     - `EasyGoPiGo3.open_left_eye` : allume l'oeil gauche de la couleur effective
 - pour l'oeil droit :
     - `EasyGoPiGo3.set_right_eye_color` : détermine la couleur de l'oeil droit
     - `EasyGoPiGo3.close_right_eye` : éteint l'oeil droit
     - `EasyGoPiGo3.open_right_eye` : allume l'oeil droit de la couleur effective
 - pour les 2 yeux à la fois :
     - `asyGoPiGo3.set_eye_color` : détermine la couleur des deux yeux
     - `asyGoPiGo3.close_eyes` : éteint les deux yeux
     - `asyGoPiGo3.open_eyes` : allume les deux yeux de leur couleur effective
 - la couleur est déterminée par 
     - un tuple de trois entiers
     - les trois valeurs représentent dans l'ordre la quantité de : rouge-vert-bleu
     - chaque valeur est comprise dans l'intervalle [0, 255]
     - par exemple :
         - `(255, 0, 0)` pour rouge
         - `(255, 128, 0)` pour orange
         - `(255, 0, 255)` pour magenta
 
**Important** : L'utilisation des yeux est un mécanisme puissant pour donner de la rétroaction. N'oubliez de garder en tête que :
 - lorsque vous changer la couleur d'un oeil par la fonction appropriée, la couleur ne change pas; il faut appeler la fonction permettant d'ouvrir l'oeil après avoir changé sa couleur
 - les couleurs sont difficiles à discernées près du blanc ou du noir
 - le blanc est vif et peut aveugler
 - mettre la couleur noir et allumer l'oeil peut donner l'impression que l'oeil est fermé

### Démonstration

In [1]:
import easygopigo3 as gpg
import time

robot = gpg.EasyGoPiGo3()
time_to_wait = 1. # en secondes

# l'oeil gauche magenta - l'oeil droit fermé
robot.set_left_eye_color((255,0,255)) # couleur magenta
robot.open_left_eye()
time.sleep(time_to_wait)
robot.close_left_eye()

# l'oeil gauche fermé - l'oeil droit vert
robot.set_right_eye_color((0,255,0)) # couleur vert
robot.open_right_eye()
time.sleep(time_to_wait)
robot.close_right_eye()

# les 2 yeux orangés
robot.set_eye_color((255,128,0)) # couleur orangé
robot.open_eyes()
time.sleep(time_to_wait)
robot.close_eyes()

# une transition de noir à blanc pour l'oeil gauche et du rouge au bleu pour l'oeil droit
for i in range(256):
    robot.set_left_eye_color((i,i,i)) # niveau d'intensité
    robot.set_right_eye_color((i,0,255-i)) # couleur de bleu à rouge en passant par un magenta de moyenne intensité
    robot.open_eyes()
    time.sleep(0.025)
    
time.sleep(time_to_wait)
robot.close_eyes()


del robot
del time
del gpg

---
### Préparation
Faites la mise en place du code commun pour cette série d'exercices

In [1]:
# Mise en place du code commun
import easygopigo3 as gpg
import time
import math

def perf_sleep(for_n_seconds : int = 0) -> None:
    time_start = time.perf_counter()
    time_counter = 0
    time_to_wait = for_n_seconds
    while (time_counter < time_to_wait):
        time_counter = time.perf_counter() - time_start
        
class Color:
    def __init__(self, r:int=255, g:int=255, b:int=255):
        self.rgb = tuple((r, g, b))
        
    def blend(self, other_color, sin_time):
        blended_color = []
        for i in range(3):
            blended_color.append(round(self.rgb[i]*sin_time + other_color[i]*(1.0-sin_time)))
        return tuple(blended_color)
    
class Brain:
    def __init__(self):
        self.logic = gpg.EasyGoPiGo3()
        self.eyes = {"left" : Eye("left", self) , "right" : Eye("right", self)}
    
    def activate_gyros(self, left_color:tuple, right_color:tuple, duration:int=30):
        time_start = time.perf_counter()
        time_counter = 0

        while time_counter < duration:
            time_counter = time.perf_counter() - time_start
            sin_time = (math.sin(time_counter * math.tau) * 0.5) + 0.5
            color1 = right_color.blend(left_color.rgb, sin_time)
            self.eyes["left"].change_color(color1)
            color2 = left_color.blend(right_color.rgb, sin_time)
            self.eyes["right"].change_color(color2)
            
        self.logic.close_eyes()
        
    def activate_kiit(self):
        pass

class Eye:
    def __init__(self, side:str, brain:Brain):
        self.brain = brain
        self.side = side
        self.func_dict = {"left" : { "set" : self.brain.logic.set_left_eye_color, 
                                    "open" : self.brain.logic.open_left_eye, 
                                    "close" : self.brain.logic.close_left_eye}, 
                          "right" : { "set" : self.brain.logic.set_right_eye_color,
                                     "open" : self.brain.logic.open_right_eye,
                                     "close" : self.brain.logic.close_right_eye}}
       
        
    def change_color(self, color:tuple):
        self.func_dict[self.side]["set"](color)
        self.func_dict[self.side]["open"]()        

brain = Brain()


---
### Exercice 6.1.
Faites apparaître sur les deux yeux à des intervals de 1.5 seconde chacune des 6 couleurs primaires et secondaires selon ce patron : rouge - jaune - vert - cyan - bleu - magenta.

Assurez-vous de fermer les yeux en quittant.

In [27]:
robot = gpg.EasyGoPiGo3()

robot.set_left_eye_color((255,0,255)) # couleur magenta
robot.open_left_eye()
robot.close_eyes()

---
### Exercice 6.2.
Selon les instructions données par la télécommande, faites les opérations suivantes sur le robot :
- `*` les opérations suivantes s'appliquent sur l'oeil gauche
- `0` les opérations suivantes s'appliquent sur les 2 yeux à la fois (c'est la situation par défaut)
- `#` les opérations suivantes s'appliquent sur l'oeil droit
- `1` plus de rouge (incrément de 32 - maximum 255)
- `2` plus de vert (incrément de 32 - maximum 255)
- `3` plus de bleu (incrément de 32 - maximum 255)
- `4` mettre le rouge au centre (128)
- `5` mettre le vert au centre (128)
- `6` mettre le bleu au centre (128)
- `7` moins de rouge (décrément de 32 - minimum 0)
- `8` moins de vert (décrément de 32 - minimum 0)
- `9` moins de bleu (décrément de 32 - minimum 0)
- `ok` terminer le programme

Initialement, les yeux sont ouverts. L'oeil gauche est rouge alors que l'oeil droit est bleu.

Les opérations suivantes sont en options pour les étudiants intéressés à explorer un autre espace de couleur. Les manipulations antérieures se font dans l'espace `RGB` alors qu'on propose ici de considérer la couleur dans l'espace `HSL` ou `HSV` à votre convenance :
- `left` change la teinte dans le sens antihoraire de 12.5 %
- `right` change la teinte dans le sens horaire de 12.5 %
- `up` plus de lumière
- `down` plus sombre

Assurez-vous de fermer les yeux en quittant.

In [ ]:
# Solution
class RemoteControl:
    def __init__(self, remote_pattern:dict)
        self.remote_pattern = remote_pattern
        
        
        
remote_pattern = { 0 : robot.forward,
                  1 : robot.backward,
                  2 : robot.right,
                  3 : robot.left,
                  4 : robot.stop }

---
### Exercice 6.3.
Réalisez un effet visuel similaire aux gyrophares des voitures de police modernes. À chaque seconde, faites passer chaque oeil d'un rouge vers le bleu avec un effet de fondu pour reprendre ensuite en sens inverse du bleu vers le rouge. Les deux yeux doivent être en alternance.

Faites cette opération pour 30 secondes et assurez-vous de fermer les yeux en quittant.

In [2]:
color1 = Color(255,0,0)
color2 = Color(0,0,255)
brain.activate_gyros(color1, color2, 4)

---
### Exercice 6.4.
Faites un jeu de lumière comme la voiture _KITT_ de la série télévisée _Knight Rider_ (voir [ce petit vidéo](https://www.youtube.com/watch?v=WxE2xWZNfOc)). Évidemment, le vôtre sera très sobre avec seulement deux lumières.

In [5]:
# Solution
brain.activate_gyros(Color(255,0,0), Color(0,0,0), 5)
brain.logic.blinker_on(0)

---
### Exercice 6.5.
Vous avez certainement remarqué que la librairie EasyGoPiGo3 offre plusieurs fonctionnalités pour modifier l'état du robot mais très peu pour connaître dans quel état il se trouve. C'est notamment le cas des yeux. Il est impossible de connaître leur couleur ou s'ils sont ouverts ou fermés.

Faites une classe qui encapsule l'utilisation d'**un seul** oeil (permettant ainsi de connaître l'état de l'oeil). De plus, on veut améliorer le comportement de la classe EasyGoPiGo3 en allumant l'oeil lorsque sa couleur est modifiée. Finalement, on doit passer au constructeur :
 - le robot
 - une information indicant si cette instance représente l'oeil droit ou l'oeil gauche - gauche par défaut
 - l'état initial de l'oeil (ouvert ou fermé) - fermé par défaut
 - la couleur initiale de l'oeil - blanc par défaut

Sur chacun des yeux, faites un effet de changement de couleur de votre cru.

N'oubliez pas de fermer les yeux à la fin.

In [ ]:
# Solution
# ...